# 📚 PDF-to-Embedding Pipeline
## Procesare 15GB Manuale Școlare → Supabase pgvector

**⚠️  IMPORTANT: Rulează celulele ÎN ORDINE (1 → 2 → 3 → 4 → 5 → 6)**

**Configurare Kaggle necesară:**
1. ✅ **Accelerator**: GPU P100 (Settings → Accelerator → GPU)
2. ✅ **Internet**: ON (Settings → Internet → ON)
3. ✅ **Secrets**: Add in Settings → Secrets:
   - `SUPABASE_URL` = your-project-url.supabase.co
   - `SUPABASE_ANON_KEY` = eyJhbGc... (anon public key)
4. ✅ **Dataset**: Upload `materiale_didactice.zip` ca Kaggle Dataset

**Timp estimat:** 18-25 ore pentru 15GB PDFs

## Cell 1: Verificare GPU + Instalare Dependențe

In [ ]:
# Verificare GPU disponibil
print("=" * 70)
print("VERIFICARE GPU")
print("=" * 70)
!nvidia-smi

print("\n" + "=" * 70)
print("INSTALARE DEPENDENȚE")
print("=" * 70)

# Instalare toate dependențele necesare
!pip install -q PyMuPDF>=1.23.0 \
    sentence-transformers>=2.2.2 \
    supabase>=2.0.0 \
    numpy>=1.24.0 \
    tqdm>=4.65.0 \
    pyyaml>=6.0

# PaddleOCR - OPȚIONAL (comentează dacă nu vrei OCR)
# OCR adaugă timp de procesare dar extrage text din imagini
ENABLE_OCR = False  # Schimbă în True pentru OCR

if ENABLE_OCR:
    print("\nInstalare PaddleOCR...")
    !pip install -q paddleocr>=2.7.0
    print("✅ PaddleOCR instalat")
else:
    print("\n⚠️  OCR DISABLED - doar text extraction simplu")
    print("    (Setează ENABLE_OCR=True pentru OCR)")

print("\n" + "=" * 70)
print("✅ TOATE DEPENDENȚELE INSTALATE")
print("=" * 70)

## Cell 2: Import Module + Configurare

In [ ]:
# Import standard libraries
import os
import sys
import logging
import time
import hashlib
import re
import json
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Tuple

# Import third-party libraries
import numpy as np
from tqdm import tqdm
import fitz  # PyMuPDF
from sentence_transformers import SentenceTransformer
from supabase import create_client
from kaggle_secrets import UserSecretsClient

# Import PaddleOCR dacă activat
if ENABLE_OCR:
    from paddleocr import PaddleOCR
    print("✅ PaddleOCR importat")

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("=" * 70)
print("ÎNCĂRCARE CREDENȚIALE SUPABASE")
print("=" * 70)

# Obține Supabase credentials din Kaggle Secrets
secret = UserSecretsClient()

try:
    SUPABASE_URL = secret.get_secret('SUPABASE_URL')
    SUPABASE_KEY = secret.get_secret('SUPABASE_ANON_KEY')
    logger.info(f"✅ Credentials loaded: {SUPABASE_URL[:40]}...")
except Exception as e:
    logger.error(f"❌ EROARE: Nu pot încărca secrets!")
    logger.error(f"   {e}")
    logger.error("\nAsigură-te că ai adăugat în Settings → Secrets:")
    logger.error("   - SUPABASE_URL")
    logger.error("   - SUPABASE_ANON_KEY")
    raise

# Configurare procesare
CONFIG = {
    'chunk_size': 500,
    'chunk_overlap': 50,
    'min_chunk_length': 50,
    'embedding_model': 'paraphrase-multilingual-mpnet-base-v2',
    'embedding_batch_size': 128,
    'embedding_device': 'cuda',  # GPU
    'supabase_batch_size': 5000,  # 5k vectors per batch
    'max_retries': 3,
    'ocr_enabled': ENABLE_OCR,
    'ocr_languages': ['ro', 'en'],
    'ocr_min_confidence': 0.6,
    'test_mode': False  # ⚠️ SCHIMBĂ ÎN True pentru testare rapidă (10 PDFs)
}

logger.info("\n" + "=" * 70)
logger.info("CONFIGURARE")
logger.info("=" * 70)
logger.info(f"Chunk size: {CONFIG['chunk_size']} caractere")
logger.info(f"Overlap: {CONFIG['chunk_overlap']} caractere")
logger.info(f"Embedding model: {CONFIG['embedding_model']}")
logger.info(f"Device: {CONFIG['embedding_device']}")
logger.info(f"Batch upload: {CONFIG['supabase_batch_size']} vectors")
logger.info(f"OCR enabled: {CONFIG['ocr_enabled']}")
logger.info(f"Test mode: {CONFIG['test_mode']} {'(doar 10 PDFs!)' if CONFIG['test_mode'] else '(toate PDFs)'}")
logger.info("=" * 70)

print("\n✅ Import și configurare complete!")

## Cell 3: Găsire PDFs în Dataset

In [ ]:
print("=" * 70)
print("CĂUTARE PDFs ÎN DATASET")
print("=" * 70)

# Localizare dataset folder în Kaggle
kaggle_input = Path('/kaggle/input')

# Listare toate dataset-urile disponibile
dataset_folders = [d for d in kaggle_input.iterdir() if d.is_dir()]
logger.info(f"\nDataset-uri găsite în /kaggle/input/:")
for folder in dataset_folders:
    logger.info(f"  - {folder.name}")

if not dataset_folders:
    logger.error("\n❌ EROARE: Nu există dataset-uri în /kaggle/input/")
    logger.error("\nTrebuie să:")
    logger.error("  1. Uploadezi materiale_didactice.zip ca Kaggle Dataset")
    logger.error("  2. Ataș dataset-ul la acest notebook (Add Data → Your Datasets)")
    logger.error("  3. Restart notebook după atașare")
    raise FileNotFoundError("Dataset lipsește!")

# Folosește primul dataset (sau specifică manual)
pdf_folder = dataset_folders[0]
logger.info(f"\n✅ Folosesc dataset: {pdf_folder.name}")

# Găsește toate PDFs recursiv
logger.info("\nCăutare PDFs recursiv...")
all_pdfs = list(pdf_folder.glob('**/*.pdf'))

if not all_pdfs:
    logger.error("\n❌ EROARE: Nu s-au găsit PDFs în dataset!")
    logger.error(f"   Verifică că dataset-ul conține foldere cu PDFs")
    raise FileNotFoundError("PDFs lipsesc din dataset!")

logger.info(f"\n" + "=" * 70)
logger.info(f"✅ GĂSITE {len(all_pdfs)} PDFs")
logger.info("=" * 70)

# Afișare sample PDFs
logger.info("\nPrime 10 PDFs:")
for i, pdf in enumerate(all_pdfs[:10]):
    size_mb = pdf.stat().st_size / 1024 / 1024
    relative_path = pdf.relative_to(pdf_folder)
    logger.info(f"  {i+1}. {relative_path} ({size_mb:.1f} MB)")

if len(all_pdfs) > 10:
    logger.info(f"  ... și încă {len(all_pdfs) - 10} PDFs")

# Calculare dimensiune totală
total_size_bytes = sum(p.stat().st_size for p in all_pdfs)
total_size_gb = total_size_bytes / 1024 / 1024 / 1024

logger.info(f"\nDimensiune totală dataset: {total_size_gb:.2f} GB")
logger.info(f"Timp estimat procesare: {len(all_pdfs) * 3 / 60:.0f}-{len(all_pdfs) * 5 / 60:.0f} ore")

print("\n✅ PDFs găsite și verificate!")

## Cell 4: Definire Funcții de Procesare

In [ ]:
print("=" * 70)
print("DEFINIRE FUNCȚII DE PROCESARE")
print("=" * 70)

# ===================================================================
# 1. PDF TEXT EXTRACTION
# ===================================================================

def extract_text_from_pdf(pdf_path: Path) -> Dict:
    """
    Extrage text din PDF folosind PyMuPDF.
    
    Returns:
        dict cu 'text', 'pages', 'images', 'status'
    """
    try:
        doc = fitz.open(pdf_path)
        full_text = ""
        images_found = []
        
        for page_num, page in enumerate(doc):
            # Extract text
            page_text = page.get_text()
            full_text += page_text + "\n"
            
            # Detect images (opțional - pentru OCR)
            if CONFIG['ocr_enabled']:
                image_list = page.get_images(full=True)
                for img in image_list:
                    images_found.append({
                        'page': page_num + 1,
                        'xref': img[0]
                    })
        
        doc.close()
        
        return {
            'text': full_text,
            'pages': len(doc),
            'images': images_found,
            'status': 'success',
            'char_count': len(full_text)
        }
        
    except Exception as e:
        return {
            'text': '',
            'pages': 0,
            'images': [],
            'status': 'error',
            'error': str(e)
        }

# ===================================================================
# 2. OCR PROCESSING (OPȚIONAL)
# ===================================================================

def extract_text_from_images_ocr(pdf_path: Path, images: List[Dict]) -> str:
    """
    Extrage text din imagini folosind PaddleOCR.
    Rulează doar dacă OCR activat.
    
    Returns:
        str - text extras din imagini
    """
    if not CONFIG['ocr_enabled']:
        return ""
    
    try:
        # Inițializare OCR (lazy loading)
        if not hasattr(extract_text_from_images_ocr, 'ocr'):
            logger.info("Inițializare PaddleOCR...")
            extract_text_from_images_ocr.ocr = PaddleOCR(
                use_angle_cls=True,
                lang='en',  # Ro + En
                use_gpu=True,
                show_log=False
            )
        
        ocr = extract_text_from_images_ocr.ocr
        ocr_text = ""
        
        # Process doar primele 5 imagini (pentru viteză)
        for img_info in images[:5]:
            try:
                # Extract image din PDF
                doc = fitz.open(pdf_path)
                page = doc[img_info['page'] - 1]
                pix = page.get_pixmap()
                img_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(
                    pix.height, pix.width, pix.n
                )
                doc.close()
                
                # Run OCR
                result = ocr.ocr(img_array, cls=True)
                
                if result and result[0]:
                    for line in result[0]:
                        text = line[1][0]
                        confidence = line[1][1]
                        if confidence >= CONFIG['ocr_min_confidence']:
                            ocr_text += text + " "
            except:
                continue
        
        return ocr_text
        
    except Exception as e:
        logger.warning(f"OCR failed: {e}")
        return ""

# ===================================================================
# 3. TEXT CHUNKING
# ===================================================================

def chunk_text(text: str) -> List[str]:
    """
    Împarte text în chunks cu overlap.
    
    Returns:
        List[str] - lista de chunks
    """
    chunk_size = CONFIG['chunk_size']
    overlap = CONFIG['chunk_overlap']
    min_length = CONFIG['min_chunk_length']
    
    # Split pe sentințe (aproximativ)
    sentences = re.split(r'(?<=[.!?])\s+', text)
    
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= chunk_size:
            current_chunk += " " + sentence if current_chunk else sentence
        else:
            if len(current_chunk) >= min_length:
                chunks.append(current_chunk.strip())
            current_chunk = sentence
    
    # Adaugă ultimul chunk
    if len(current_chunk) >= min_length:
        chunks.append(current_chunk.strip())
    
    return chunks

# ===================================================================
# 4. DEDUPLICATION
# ===================================================================

def deduplicate_chunks(chunks: List[str]) -> List[str]:
    """
    Elimină duplicate chunks (headers/footers repetate).
    
    Returns:
        List[str] - chunks unice
    """
    seen_hashes = set()
    unique_chunks = []
    
    for chunk in chunks:
        chunk_hash = hashlib.md5(chunk.encode('utf-8')).hexdigest()
        
        if chunk_hash not in seen_hashes:
            seen_hashes.add(chunk_hash)
            unique_chunks.append(chunk)
    
    return unique_chunks

# ===================================================================
# 5. EMBEDDING GENERATION
# ===================================================================

def init_embedding_model():
    """
    Inițializare sentence-transformers model.
    """
    logger.info(f"Încărcare model: {CONFIG['embedding_model']}")
    logger.info(f"Device: {CONFIG['embedding_device']}")
    
    model = SentenceTransformer(
        CONFIG['embedding_model'],
        device=CONFIG['embedding_device']
    )
    
    logger.info(f"✅ Model încărcat: 768 dimensiuni")
    return model

def generate_embeddings(texts: List[str], model) -> np.ndarray:
    """
    Generează embeddings pentru listă de texte.
    
    Returns:
        np.ndarray shape (N, 768)
    """
    embeddings = model.encode(
        texts,
        batch_size=CONFIG['embedding_batch_size'],
        convert_to_numpy=True,
        show_progress_bar=False
    )
    return embeddings

# ===================================================================
# 6. SUPABASE UPLOAD
# ===================================================================

def init_supabase():
    """
    Inițializare Supabase client.
    """
    client = create_client(SUPABASE_URL, SUPABASE_KEY)
    logger.info("✅ Supabase client inițializat")
    return client

def upload_vectors_to_supabase(supabase, vectors: List[Dict]) -> Dict:
    """
    Upload vectors în Supabase cu batching.
    
    Returns:
        dict cu 'success' și 'failed' counts
    """
    batch_size = CONFIG['supabase_batch_size']
    max_retries = CONFIG['max_retries']
    
    total_success = 0
    total_failed = 0
    
    for i in range(0, len(vectors), batch_size):
        batch = vectors[i:i + batch_size]
        
        # Retry logic
        for attempt in range(max_retries):
            try:
                response = supabase.table('document_embeddings').insert(batch).execute()
                total_success += len(batch)
                break  # Success, exit retry loop
            except Exception as e:
                if attempt == max_retries - 1:
                    logger.warning(f"Batch {i//batch_size} failed after {max_retries} retries: {e}")
                    total_failed += len(batch)
                else:
                    time.sleep(2)  # Wait before retry
    
    return {'success': total_success, 'failed': total_failed}

logger.info("\n✅ Toate funcțiile definite!")
logger.info("   - extract_text_from_pdf()")
logger.info("   - extract_text_from_images_ocr() [opțional]")
logger.info("   - chunk_text()")
logger.info("   - deduplicate_chunks()")
logger.info("   - init_embedding_model()")
logger.info("   - generate_embeddings()")
logger.info("   - upload_vectors_to_supabase()")
print("=" * 70)

## Cell 5: Inițializare Componente (Model + Supabase)

In [ ]:
print("=" * 70)
print("INIȚIALIZARE COMPONENTE")
print("=" * 70)

# Inițializare embedding model
logger.info("\nÎncarc embedding model pe GPU...")
embedding_model = init_embedding_model()

# Inițializare Supabase
logger.info("\nConectare la Supabase...")
supabase = init_supabase()

# Testare conexiune Supabase
try:
    test_response = supabase.table('document_embeddings').select('count', count='exact').limit(1).execute()
    current_count = test_response.count if test_response.count else 0
    logger.info(f"✅ Supabase conectat - {current_count} vectors existenți în DB")
except Exception as e:
    logger.error(f"❌ Eroare testare conexiune Supabase: {e}")
    logger.error("Verifică că ai rulat supabase_setup.sql în SQL Editor!")
    raise

print("\n" + "=" * 70)
print("✅ COMPONENTE INIȚIALIZATE - GATA DE PROCESARE")
print("=" * 70)

## Cell 6: PIPELINE PRINCIPAL DE PROCESARE

**⚠️  IMPORTANT**: Această celulă depinde de celulele anterioare!
- Cell 2: `CONFIG`
- Cell 3: `all_pdfs`, `pdf_folder`
- Cell 5: `embedding_model`, `supabase`

Dacă primești `NameError`, rulează celulele 1-5 înainte!

In [ ]:
print("\n" + "=" * 70)
print("ÎNCEPE PROCESAREA PRINCIPALĂ")
print("=" * 70)

# ===================================================================
# VERIFICARE DEPENDENȚE DIN CELULELE ANTERIOARE
# ===================================================================
required_vars = {
    'all_pdfs': 'Cell 3 (Găsire PDFs)',
    'pdf_folder': 'Cell 3 (Găsire PDFs)',
    'embedding_model': 'Cell 5 (Inițializare Componente)',
    'supabase': 'Cell 5 (Inițializare Componente)',
    'CONFIG': 'Cell 2 (Configurare)'
}

missing_vars = []
for var_name, source_cell in required_vars.items():
    if var_name not in globals():
        missing_vars.append(f"  ❌ {var_name} (din {source_cell})")

if missing_vars:
    print("\n" + "=" * 70)
    print("❌ EROARE: Lipsesc variabile necesare!")
    print("=" * 70)
    print("\nRulează celulele în ordine ÎNAINTE de această celulă:\n")
    for missing in missing_vars:
        print(missing)
    print("\nPaşi de rezolvare:")
    print("  1. Rulează Cell 1 (Instalare Dependențe)")
    print("  2. Rulează Cell 2 (Configurare)")
    print("  3. Rulează Cell 3 (Găsire PDFs)")
    print("  4. Rulează Cell 4 (Definire Funcții)")
    print("  5. Rulează Cell 5 (Inițializare Componente)")
    print("  6. Apoi rulează această celulă\n")
    raise NameError(f"Variabile lipsă: {', '.join([v.split(' ')[1] for v in missing_vars])}")

print("✅ Toate dependențele verificate - continuăm procesarea...\n")

# ===================================================================
# MOD TEST vs COMPLET (din CONFIG)
# ===================================================================
if CONFIG['test_mode']:
    pdfs_to_process = all_pdfs[:10]
    logger.warning("\n⚠️  MOD TEST ACTIVAT - se procesează doar 10 PDFs!")
    logger.warning("   Schimbă CONFIG['test_mode']=False în Cell 2 pentru procesare completă\n")
else:
    pdfs_to_process = all_pdfs
    logger.info(f"\nMod COMPLET - se procesează toate {len(all_pdfs)} PDFs\n")

# Inițializare statistici
stats = {
    'start_time': time.time(),
    'processed_pdfs': 0,
    'failed_pdfs': 0,
    'total_pages': 0,
    'total_chars': 0,
    'total_chunks': 0,
    'total_unique_chunks': 0,
    'total_embeddings': 0,
    'uploaded_vectors': 0,
    'failed_uploads': 0,
    'ocr_images_processed': 0
}

# Buffer pentru batch upload
upload_buffer = []
buffer_max_size = CONFIG['supabase_batch_size']

# ===================================================================
# MAIN PROCESSING LOOP
# ===================================================================

logger.info("Începe procesare...\n")

with tqdm(total=len(pdfs_to_process), desc="📄 Procesare PDFs") as pbar:
    
    for pdf_path in pdfs_to_process:
        try:
            # ============================================================
            # STEP 1: EXTRAGERE TEXT DIN PDF
            # ============================================================
            result = extract_text_from_pdf(pdf_path)
            
            if result['status'] != 'success':
                logger.warning(f"Failed: {pdf_path.name} - {result.get('error', 'Unknown error')}")
                stats['failed_pdfs'] += 1
                pbar.update(1)
                continue
            
            text = result['text']
            pages = result['pages']
            images = result['images']
            
            stats['processed_pdfs'] += 1
            stats['total_pages'] += pages
            stats['total_chars'] += len(text)
            
            # ============================================================
            # STEP 2: OCR PE IMAGINI (dacă activat)
            # ============================================================
            ocr_text = ""
            if CONFIG['ocr_enabled'] and images:
                ocr_text = extract_text_from_images_ocr(pdf_path, images)
                if ocr_text:
                    text = text + "\n" + ocr_text
                    stats['ocr_images_processed'] += len(images)
            
            # Skip PDFs fără text
            if len(text.strip()) < 100:
                logger.warning(f"Skipping {pdf_path.name} - prea puțin text ({len(text)} chars)")
                pbar.update(1)
                continue
            
            # ============================================================
            # STEP 3: CHUNKING TEXT
            # ============================================================
            chunks = chunk_text(text)
            stats['total_chunks'] += len(chunks)
            
            # ============================================================
            # STEP 4: DEDUPLICARE CHUNKS
            # ============================================================
            unique_chunks = deduplicate_chunks(chunks)
            stats['total_unique_chunks'] += len(unique_chunks)
            
            # ============================================================
            # STEP 5: GENERARE EMBEDDINGS
            # ============================================================
            embeddings = generate_embeddings(unique_chunks, embedding_model)
            stats['total_embeddings'] += len(embeddings)
            
            # ============================================================
            # STEP 6: PREGĂTIRE PENTRU UPLOAD
            # ============================================================
            pdf_relative_path = str(pdf_path.relative_to(pdf_folder))
            
            # Extract metadata din path (ex: clasa_2/matematica/...)
            path_parts = pdf_relative_path.split(os.sep)
            clasa = 0
            materie = "General"
            
            # Încearcă să extragi clasa din path (ex: clasa_2)
            for part in path_parts:
                if 'clasa' in part.lower():
                    try:
                        clasa = int(re.search(r'\d+', part).group())
                    except:
                        pass
                # Materia poate fi numele folderului
                if len(path_parts) > 1:
                    materie = path_parts[1] if len(path_parts) > 2 else path_parts[0]
            
            # Creează vector items pentru Supabase
            for idx, (chunk, embedding) in enumerate(zip(unique_chunks, embeddings)):
                chunk_hash = hashlib.md5(chunk.encode('utf-8')).hexdigest()
                
                vector_item = {
                    'chunk_id': f"{pdf_path.stem}_{idx}_{chunk_hash[:8]}",
                    'text': chunk[:10000],  # Limită 10k chars pentru DB
                    'embedding': embedding.tolist(),
                    'source_pdf': pdf_relative_path,
                    'page_num': 1,  # Simplificat - nu trackuim pagina exactă
                    'clasa': clasa,
                    'materie': materie,
                    'capitol': 'General',
                    'chunk_hash': chunk_hash,
                    'has_images': len(images) > 0
                }
                
                upload_buffer.append(vector_item)
            
            # ============================================================
            # STEP 7: UPLOAD BATCH CÂND BUFFER PLIN
            # ============================================================
            if len(upload_buffer) >= buffer_max_size:
                upload_result = upload_vectors_to_supabase(supabase, upload_buffer)
                stats['uploaded_vectors'] += upload_result['success']
                stats['failed_uploads'] += upload_result['failed']
                upload_buffer = []  # Clear buffer
            
            pbar.update(1)
            
        except Exception as e:
            logger.error(f"Eroare critică procesare {pdf_path.name}: {e}")
            stats['failed_pdfs'] += 1
            pbar.update(1)
            continue

# ===================================================================
# UPLOAD ULTIMUL BATCH RĂMAS ÎN BUFFER
# ===================================================================
if upload_buffer:
    logger.info(f"\nUpload ultimul batch: {len(upload_buffer)} vectors...")
    upload_result = upload_vectors_to_supabase(supabase, upload_buffer)
    stats['uploaded_vectors'] += upload_result['success']
    stats['failed_uploads'] += upload_result['failed']

stats['end_time'] = time.time()
stats['elapsed_seconds'] = stats['end_time'] - stats['start_time']

print("\n" + "=" * 70)
print("✅ PROCESARE COMPLETĂ")
print("=" * 70)

## Cell 7: Afișare Statistici Finale

In [ ]:
print("\n" + "=" * 70)
print("📊 STATISTICI FINALE")
print("=" * 70)

# Calculare timp
elapsed_hours = stats['elapsed_seconds'] / 3600
elapsed_minutes = (stats['elapsed_seconds'] % 3600) / 60

print(f"\n⏱️  TIMP PROCESARE")
print(f"   Total: {elapsed_hours:.1f} ore ({elapsed_minutes:.0f} minute)")
print(f"   Timp mediu per PDF: {stats['elapsed_seconds'] / max(stats['processed_pdfs'], 1):.1f} secunde")

print(f"\n📄 PDFs")
print(f"   ✅ Procesate cu succes: {stats['processed_pdfs']:,}")
print(f"   ❌ Failed: {stats['failed_pdfs']:,}")
print(f"   📑 Total pagini: {stats['total_pages']:,}")

print(f"\n📝 TEXT")
print(f"   Total caractere: {stats['total_chars']:,}")
print(f"   Mediu per PDF: {stats['total_chars'] // max(stats['processed_pdfs'], 1):,} caractere")

print(f"\n✂️  CHUNKING")
print(f"   Total chunks create: {stats['total_chunks']:,}")
print(f"   După deduplicare: {stats['total_unique_chunks']:,}")
print(f"   Duplicate eliminate: {stats['total_chunks'] - stats['total_unique_chunks']:,}")
print(f"   Mediu chunks per PDF: {stats['total_unique_chunks'] // max(stats['processed_pdfs'], 1)}")

print(f"\n🧠 EMBEDDINGS")
print(f"   Total vectors generate: {stats['total_embeddings']:,}")
print(f"   Dimensiune: 768 (paraphrase-multilingual-mpnet-base-v2)")

if CONFIG['ocr_enabled']:
    print(f"\n🔍 OCR")
    print(f"   Imagini procesate: {stats['ocr_images_processed']:,}")

print(f"\n☁️  SUPABASE UPLOAD")
print(f"   ✅ Uploaded cu succes: {stats['uploaded_vectors']:,} vectors")
print(f"   ❌ Failed uploads: {stats['failed_uploads']:,}")

# Estimare dimensiune DB
estimated_db_mb = (stats['uploaded_vectors'] * 3.5) / 1024  # ~3.5KB per vector
print(f"\n💾 DIMENSIUNE DATABASE")
print(f"   Estimat: ~{estimated_db_mb:.0f} MB")
print(f"   Supabase free tier: 500 MB")
if estimated_db_mb < 500:
    print(f"   ✅ În limită ({500 - estimated_db_mb:.0f} MB rămase)")
else:
    print(f"   ⚠️  DEPĂȘEȘTE limita cu {estimated_db_mb - 500:.0f} MB")

print("\n" + "=" * 70)
print("🎉 PROCESARE FINALIZATĂ CU SUCCES!")
print("=" * 70)

## Cell 8: Verificare Supabase Database

In [ ]:
print("\n" + "=" * 70)
print("🔍 VERIFICARE SUPABASE DATABASE")
print("=" * 70)

try:
    # Count total vectors
    response = supabase.table('document_embeddings').select('count', count='exact').limit(1).execute()
    db_count = response.count if response.count else 0
    
    print(f"\n📊 Vectors în database: {db_count:,}")
    print(f"   Uploaded în acest run: {stats['uploaded_vectors']:,}")
    
    if db_count == stats['uploaded_vectors']:
        print("   ✅ VERIFICARE PASSED - toate vectorii uploadați")
    else:
        print(f"   ⚠️  Diferență: {abs(db_count - stats['uploaded_vectors']):,} vectors")
        print("      (Poate include vectors din rulări anterioare)")
    
    # Sample query - afișează primele 3 records
    print("\n📄 Sample records din database:")
    sample = supabase.table('document_embeddings').select('chunk_id, source_pdf, clasa, materie').limit(3).execute()
    
    if sample.data:
        for i, record in enumerate(sample.data, 1):
            print(f"   {i}. {record['chunk_id']}")
            print(f"      PDF: {record['source_pdf']}")
            print(f"      Clasa: {record['clasa']}, Materie: {record['materie']}")
    else:
        print("   ⚠️  Nu s-au găsit records")
    
    print("\n✅ Verificare completă - database OK!")
    
except Exception as e:
    print(f"\n❌ Eroare verificare database: {e}")
    print("   Verifică conexiunea Supabase și tabelul 'document_embeddings'")

print("=" * 70)

## Cell 9: Test Similarity Search (Opțional)

In [ ]:
print("\n" + "=" * 70)
print("🔍 TEST SIMILARITY SEARCH")
print("=" * 70)

if stats['uploaded_vectors'] == 0:
    print("\n⚠️  Nu există vectors în database - skip search test")
else:
    try:
        # Query de test în română
        test_queries = [
            "Cum se calculează aria unui pătrat?",
            "Ce este fotosinteza?",
            "Capitala României"
        ]
        
        for query_text in test_queries:
            print(f"\n📝 Query: '{query_text}'")
            
            # Generate query embedding
            query_embedding = embedding_model.encode([query_text])[0].tolist()
            
            # Call Supabase RPC function
            try:
                results = supabase.rpc('match_documents', {
                    'query_embedding': query_embedding,
                    'match_count': 3
                }).execute()
                
                if results.data:
                    print("   Rezultate:")
                    for i, result in enumerate(results.data, 1):
                        text_preview = result.get('text', '')[:80]
                        similarity = result.get('similarity', 0)
                        source = result.get('source_pdf', 'unknown')
                        print(f"      {i}. [{similarity:.1%}] {text_preview}...")
                        print(f"         Source: {source}")
                else:
                    print("   ⚠️  Nu s-au găsit rezultate")
                    
            except Exception as e:
                print(f"   ❌ Eroare query: {e}")
                print("      (Verifică că match_documents() RPC function există în Supabase)")
                print("      (Rulează supabase_setup.sql pentru a crea funcția)")
                break
        
        print("\n✅ Similarity search funcționează!")
        print("   Poți folosi match_documents() RPC în aplicația ta")
        
    except Exception as e:
        print(f"\n❌ Eroare test search: {e}")

print("\n" + "=" * 70)

## Cell 10: NEXT STEPS & CLEANUP

In [ ]:
print("\n" + "=" * 70)
print("🎯 NEXT STEPS")
print("=" * 70)

print("""
✅ DONE:
   1. PDFs procesate cu succes
   2. Embeddings generate (768 dimensiuni)
   3. Vectors uploadate în Supabase

🔧 TODO (dacă nu ai făcut deja):
   
   1. CREATE HNSW INDEX în Supabase SQL Editor:
      
      CREATE INDEX IF NOT EXISTS idx_embedding_hnsw
      ON document_embeddings
      USING hnsw (embedding vector_cosine_ops)
      WITH (m = 16, ef_construction = 64);
      
      ⏱️  Timp: 30-60 minute pentru 600k vectors (one-time)
   
   2. VERIFICĂ RPC FUNCTION match_documents() există
      (Ar trebui creată din supabase_setup.sql)

🚀 FOLOSIRE ÎN AI TUTORING APP:
   
   # În aplicația ta Python/Node.js:
   
   1. User face întrebare: "Cum calculez aria unui pătrat?"
   
   2. Generate query embedding:
      model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
      query_emb = model.encode([question])[0].tolist()
   
   3. Caută în Supabase:
      results = supabase.rpc('match_documents', {
          'query_embedding': query_emb,
          'match_count': 10,
          'filter_clasa': 2  # Optional
      }).execute()
   
   4. Feed context la LLM:
      context = "\n".join([r['text'] for r in results.data])
      prompt = f"Context: {context}\n\nÎntrebare: {question}"
      response = openai.ChatCompletion.create(...)

💾 STORAGE:
   - Vectors în Supabase: PERMANENT (până la limita de 500MB)
   - Poți șterge acum PDFs locale pentru a elibera 15GB
   - Procesarea se face O SINGURĂ DATĂ

💰 COST:
   - Total: $0 (Kaggle free + Supabase free tier)
   - Mentenanță: $0/lună

""")

print("=" * 70)
print("🎉 SUCCESS! Embeddings gata de folosit în AI tutoring!")
print("=" * 70)